In [78]:
import boto3
import json
import os
import pprint

In [74]:
BUCKET = 'mediboard-textract'

In [28]:
STUDIES_PATH = os.path.expanduser('~/data/mediboard/studies/unstructured')

In [30]:
pdfs = os.listdir(STUDIES_PATH)

In [33]:
pdfs, len(pdfs), STUDIES_PATH

(['1475-2832-3-2.pdf',
  'nejmoa010534.pdf',
  'appi.ajp.161.4.677.pdf',
  '141767.pdf',
  'emslie1997.pdf',
  'ioi00178.pdf',
  '91U.S.ModafinilinNarcolepsyMulticenterStudyGroupNeurology2000.pdf',
  'appi.ajp.161.6.1079.pdf',
  'appi.ajp.157.10.1686.pdf',
  'sleep-26-7-793.pdf'],
 10,
 '/home/davon/data/mediboard/studies/unstructured')

In [54]:
import base64

with open("{}/{}".format(STUDIES_PATH, pdfs[0]), "rb") as study_file:
    encoded_string = base64.b64encode(study_file.read())
    print(type(encoded_string))

<class 'bytes'>


In [60]:
textract = boto3.client('textract', region_name='us-west-2')

In [65]:
response = textract.start_document_text_detection(
    DocumentLocation={
        'S3Object': {
            'Bucket': 'mediboard-textract',
            'Name': 'unstructured/141767.pdf',
            # 'Version': 'string'
        }
    },
    # NotificationChannel={
    #     'SNSTopicArn': 'string',
    #     'RoleArn': 'string'
    # },
    OutputConfig={
        'S3Bucket': 'mediboard-textract',
        'S3Prefix': 'structured'
    },
)

In [70]:
print(response)
job_id = response['JobId']

{'JobId': 'd594973d343cbfa32ea62ddf81e936dae1764b312b32134b4041fb8918229950', 'ResponseMetadata': {'RequestId': '998ac3cb-8de7-492d-99a8-6fd30bccc221', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '998ac3cb-8de7-492d-99a8-6fd30bccc221', 'content-type': 'application/x-amz-json-1.1', 'content-length': '76', 'date': 'Thu, 20 Oct 2022 19:17:41 GMT'}, 'RetryAttempts': 0}}


In [ ]:
# TODO(davonprewitt): Create 

In [165]:
from dateutil.parser import parse

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    # print(type(string))
    if type(string) != str:
        return False
    try: 
        # print("H", string)
        # print("i", parse(string, fuzzy=fuzzy))
        return True

    except ValueError:
        return False

In [198]:
import datetime as dt

def is_date_2(string):

    fmts = ('%d %B %Y', '%B %d %Y')
    if type(string) != str:
        return False

    print("HEY")
    for fmt in fmts:
        try:
            print("BEFORE")
            t = dt.datetime.strptime(string, fmt)
            print("AFTER")
            return True
        except ValueError as err:
            pass

    return False


In [221]:
import datefinder

def is_date_3(string):
    if type(string) != str:
        return []

    matches = datefinder.find_dates(string, strict=True)
    return [match for match in matches]

In [246]:
s3 = boto3.client('s3', region_name='us-west-2')

# dict_keys(
# ['BlockType', 
# 'ColumnIndex', 
# 'ColumnSpan',
# 'Confidence', 
# 'EntityTypes', 
# 'Geometry', 
# 'Hint', 
# 'Id', 
# 'Page', 
# 'PageClassification', 
# 'Query', 
# 'Relationships', 
# 'RowIndex', 
# 'RowSpan', 
# 'SelectionStatus', 
# 'Text', 
# 'TextType'
# ])
def flatten(l):
    return [item for sublist in l for item in sublist]

for key in s3.list_objects(Bucket=BUCKET)['Contents']:
    object = key['Key']
    if 'structured/{}'.format(job_id) in object and '.s3_access_check' not in object:
        print(object)
        data = s3.get_object(Bucket=BUCKET, Key=object)
        contents = data['Body'].read()
        parsed = json.loads(contents.decode("utf-8"))
        
        title = ' '.join([p['Text'] for p in parsed['Blocks'] if p['BlockType'] == "LINE" and p['Geometry']['BoundingBox']['Height'] > .018]).lstrip('Clinical Study')
        date_pos = [is_date_3(p['Text']) for p in parsed['Blocks'] if len(is_date_3(p['Text'])) > 0]
        # print(flatten(date_pos)[-1])
        # print(title)
        for p in parsed['Blocks']:
            if (len(p['Relationships'][0]['Ids']) > 14 and 'Copyright'):
                print(p['Text'])
            # else:
            #     pass
        
        break


structured/d594973d343cbfa32ea62ddf81e936dae1764b312b32134b4041fb8918229950/1
None
Copyright © 2015 D. Wolf and J. Otto. This is an open access article distributed under the Creative Commons Attribution License,
which permits unrestricted use, distribution, and reproduction in any medium, provided the original work is properly cited.
Lidocaine is a well-accepted topical anaesthetic, also used in minors to treat painful conditions on mucosal membranes. This
generate efficacy and safety data for a lidocaine gel (2%) in younger children with painful conditions in the oral cavity. One hundred
sixty-one children were included in two subgroups: 4-8 years, average age 6.4 years, treated with verum or placebo and 6 months-
<4 years, average age 1.8 years, treated only with verum. Pain reduction was measured from the time prior to administration to 10
or 30 minutes after. In addition, adverse events and local tolerability were evaluated. In group I, pain was reduced significantly after
treatmen

TypeError: 'NoneType' object is not subscriptable